In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# 참고 사이트

https://dacon.io/en/competitions/official/235914/codeshare/5728

# 필요 라이브러리

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 72.2 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.cm as cm
import seaborn as sns
import re


from konlpy.tag import Okt
okt = Okt()
from konlpy.tag import Kkma

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DSL/labeling_total.csv", encoding= "utf-8")

In [ ]:
#label 별로 리뷰 나누기 
improve_review = df[(df['Label']==2) | (df['Label']==0)] #bad, improve 합침
good_review = df[df['Label']==1]
personal_review = df[df['Label']==9]

In [ ]:
reviews = df['Review']

In [ ]:
reviews.head(10)

0     매번 9시간 넘게 자거나 깊게 잠들지 못하고 수시로 깨는데 이 거 켜 놨을 때 딱...
1     처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한 데 이틀 ...
2     그리고 가끔 아침에 일어났을 때 머리가 몽롱 한 데 그 땐 아침밥을 먹거나 커피 ...
3     단점 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌...
4     ㅜㅠ 앱에 표시 되는 광고에 대한 평가 란이 있었는데 그런 거 신경 안 쓰고 잠잘...
5           엄청 잘 잔 거 같은데 워치에는 3 번이나 깨고 수면 점수 낮게 기록 되네요
6     근데 또 꿈결에 윗 집 진동 알람 땜에 짜증났었는데 앱 소리 였는지 뭐인지 묘 해...
7      뭐 부족한 부분이 있다는 후기가 있어서 한번 경험하고 결제 진행을 선택 하긴 어렵네요
8     개인적인 문 제일 수도 있지만 미라클 나 잇을 사용 할 때마다 휴대폰 진동에 머리...
9     그리고 수면 중지를 해도 주파수 소리가 계속 나오고 알 림 창 알림을 들어가면 이...
Name: Review, dtype: object

# 한국어 전처리

1. 한국어가 아닌 문자는 공백으로 변환 
2. 공백기준 쪼개기(토큰화)
3. stopwords 제거
4. pos tag == 명사, 동사 반환(Okt, Kkma)
5. 공백으로 구분된 문자열을 결합하여 결과 반환

In [ ]:
stopwords_list = ["!","\"","$","%","&","'","(",")","*","+",",","-",".","...","0","1","2","3","4","5","6","7","8","9",";","<","=",">","?","@","\\","^","_","`","|","~","·",
                  "—","——","‘","’","“","”","…","、","。","〈","〉","《","》","가","가까스로","가령","각","각각","각자","각종","갖고말하자면","같다","같이","개의치않고","거니와","거바","거의","것",
                  "것과 같이","것들","게다가","게우다","겨우","견지에서","결과에 이르다","결국","결론을 낼 수 있다","겸사겸사","고려하면","고로","곧","공동으로","과","과연","관계가 있다","관계없이",
                  "관련이 있다","관하여","관한","관해서는","구","구체적으로","구토하다","그","그들","그때","그래","그래도","그래서","그러나","그러니","그러니까","그러면","그러므로","그러한즉","그런 까닭에",
                  "그런데","그런즉","그럼","그럼에도 불구하고","그렇게 함으로써","그렇지","그렇지 않다면","그렇지 않으면","그렇지만","그렇지않으면","그리고","그리하여","그만이다","그에 따르는","그위에","그저",
                  "그중에서","그치지 않다","근거로","근거하여","기대여","기점으로","기준으로","기타","까닭으로","까악","까지","까지 미치다","까지도","꽈당","끙끙","끼익","나","나머지는","남들","남짓",
                  "너","너희","너희들","네","넷","년","논하지 않다","놀라다","누가 알겠는가","누구","다른","다른 방면으로","다만","다섯","다소","다수","다시 말하자면","다시말하면","다음","다음에",
                  "다음으로","단지","답다","당신","당장","대로 하다","대하면","대하여","대해 말하자면","대해서","댕그","더구나","더군다나","더라도","더불어","더욱더","더욱이는","도달하다","도착하다",
                  "동시에","동안","된바에야","된이상","두번째로","둘","둥둥","뒤따라","뒤이어","든간에","들","등","등등","딩동","따라","따라서","따위","따지지 않다","딱","때","때가 되어","때문에","또","또한","뚝뚝","라 해도","령","로","로 인하여","로부터","로써","륙","를","마음대로","마저","마저도","마치","막론하고","만 못하다","만약","만약에","만은 아니다","만이 아니다","만일","만큼","말하자면","말할것도 없고","매","매번","메쓰겁다","몇","모","모두","무렵","무릎쓰고","무슨","무엇","무엇때문에","물론","및","바꾸어말하면","바꾸어말하자면","바꾸어서 말하면","바꾸어서 한다면","바꿔 말하면","바로","바와같이","밖에 안된다","반대로","반대로 말하자면","반드시","버금","보는데서","보다더","보드득","본대로","봐","봐라","부류의 사람들","부터","불구하고","불문하고","붕붕","비걱거리다","비교적","비길수 없다","비로소","비록","비슷하다","비추어 보아","비하면","뿐만 아니라","뿐만아니라","뿐이다","삐걱","삐걱거리다","사","삼","상대적으로 말하자면","생각한대로","설령","설마","설사","셋","소생","소인","솨","쉿","습니까","습니다","시각","시간","시작하여","시초에","시키다","실로","심지어","아","아니","아니나다를가","아니라면","아니면","아니었다면","아래윗","아무거나","아무도","아야","아울러","아이","아이고","아이구","아이야","아이쿠","아하","아홉","안 그러면","않기 위하여","않기 위해서","알 수 있다","알았어","앗","앞에서","앞의것","야","약간","양자","어","어기여차","어느","어느 년도","어느것","어느곳","어느때","어느쪽","어느해","어디","어때","어떠한","어떤","어떤것","어떤것들","어떻게","어떻해","어이","어째서","어쨋든","어쩔수 없다","어찌","어찌됏든","어찌됏어","어찌하든지","어찌하여","언제","언젠가","얼마","얼마 안 되는 것","얼마간","얼마나","얼마든지","얼마만큼","얼마큼","엉엉","에","에 가서","에 달려 있다","에 대해","에 있다","에 한하다","에게","에서","여","여기","여덟","여러분","여보시오","여부","여섯","여전히","여차","연관되다","연이서","영","영차","옆사람","예","예를 들면","예를 들자면","예컨대","예하면","오","오로지","오르다","오자마자","오직","오호","오히려","와","와 같은 사람들","와르르","와아","왜","왜냐하면","외에도","요만큼","요만한 것","요만한걸","요컨대","우르르","우리","우리들","우선","우에 종합한것과같이","운운","월","위에서 서술한바와같이","위하여","위해서","윙윙","육","으로","으로 인하여","으로서","으로써","을","응","응당","의","의거하여","의지하여","의해","의해되다","의해서","이","이 되다","이 때문에","이 밖에","이 외에","이 정도의","이것","이곳","이때","이라면","이래","이러이러하다","이러한","이런","이럴정도로","이렇게 많은 것","이렇게되면","이렇게말하자면","이렇구나","이로 인하여","이르기까지","이리하여","이만큼","이번","이봐","이상","이어서","이었다","이와 같다","이와 같은","이와 반대로","이와같다면","이외에도","이용하여","이유만으로","이젠","이지만","이쪽","이천구","이천육","이천칠","이천팔","인 듯하다","인젠","일","일것이다","일곱","일단","일때","일반적으로","일지라도","임에 틀림없다","입각하여","입장에서","잇따라","있다","자","자기","자기집","자마자","자신","잠깐","잠시","저","저것","저것만큼","저기","저쪽","저희","전부","전자","전후","점에서 보아","정도에 이르다","제","제각기","제외하고","조금","조차","조차도","졸졸","좀","좋아","좍좍","주룩주룩","주저하지 않고","줄은 몰랏다","줄은모른다","중에서","중의하나","즈음하여","즉","즉시","지든지","지만","지말고","진짜로","쪽으로","차라리","참","참나","첫번째로","쳇","총적으로","총적으로 말하면","총적으로 보면","칠","콸콸","쾅쾅","쿵","타다","타인","탕탕","토하다","통하여","툭","퉤","틈타","팍","팔","퍽","펄렁","하","하게될것이다","하게하다","하겠는가","하고 있다","하고있었다","하곤하였다","하구나","하기 때문에","하기 위하여","하기는한데","하기만 하면","하기보다는","하기에","하나","하느니","하는 김에","하는 편이 낫다","하는것도","하는것만 못하다","하는것이 낫다","하는바","하더라도","하도다","하도록시키다","하도록하다","하든지","하려고하다","하마터면","하면 할수록","하면된다","하면서","하물며","하여금","하여야","하자마자","하지 않는다면","하지 않도록","하지마","하지마라","하지만","하하","한 까닭에","한 이유는","한 후","한다면","한다면 몰라도","한데","한마디","한적이있다","한켠으로는","한항목","할 따름이다","할 생각이다","할 줄 안다","할 지경이다","할 힘이 있다","할때","할만하다","할망정","할뿐","할수있다","할수있어","할줄알다","할지라도","할지언정","함께","해도된다","해도좋다","해봐요","해서는 안된다","해야한다","해요","했어요","향하다","향하여","향해서","허","허걱","허허","헉","헉헉","헐떡헐떡","형식으로 쓰여","혹시","혹은","혼자","훨씬","휘익","휴","흐흐","흥","힘입어","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８","９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥",'ㅜㅠ','ㅎㅎ','ㅠㅠ','ㅎㅎㅎ','ㅋㅋㅋ','ㅋㅋ']

In [ ]:
#전처리 함수
def to_words(corpus):
  korean_only = re.sub('^[가-힣]','',corpus)
  words = korean_only.split()
  stops = set(stopwords_list)
  meaningful_words = [w for w in words if not w in stops]
  mystring = ''.join(map(str,meaningful_words))
  nouns = okt.nouns(mystring)
  nounstring = ','.join(map(str,nouns))
  return nounstring

In [ ]:
improve = improve_review['Review'].apply(to_words)
good = good_review['Review'].apply(to_words)
personal = personal_review['Review'].apply(to_words)
#각 라벨별로 전처리 함수 돌리기

In [ ]:
personal

1                                처음,휴대폰,소리,적응,머리,이틀,적응,아집
4                앱,표시,광고,대한,평가,란,신경안,때,만사,앱,광고,평가,란,보고,처음
5                                              워치,번,점수,기록
6                          꿈결,집,진동,알람,땜,앱,소리,뭐,묘아,직,베타,단계
11      오류,버그,중지,이용권,다시,드릴,필요,이용자,입장,버그,문제,이용권,문제,월간,플...
                              ...                        
7203                                        기와,달,무료,체험,후연
7206                                   거미,리,결제,체험,시작,결제,임
7229                                       전자파,몸,머리,직빵,생각
7255                               무료,체험,안,건너,기하,알람,기능,사용
7314                                   구매,프로모션,코드,프리미엄,등록
Name: Review, Length: 615, dtype: object

In [ ]:
#list 로 만들기(imporve_word)
improve_word = []
for word in improve:
  improve_word.append(word) 

In [ ]:
good_word = []
for word in good:
  good_word.append(word) 

In [ ]:
personal_word = []
for word in personal:
  personal_word.append(word) 

## Bertopic

In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 8.3 MB/s 
     |████████████████████████████████| 85 kB 5.4 MB/s 
     |████████████████████████████████| 636 kB 29.4 MB/s 
     |████████████████████████████████| 5.2 MB 46.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 8.7 MB/s 
     |████████████████████████████████| 4.9 MB 50.1 MB/s 
     |████████████████████████████████| 1.3 MB 50.7 MB/s 
     |████████████████████████████████| 163 kB 70.5 MB/s 
     |████████████████████████████████| 6.6 MB 52.6 MB/s 
     |████████████████████████████████| 1.1 MB 58.2 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2340268 sha256=a2579a73f425ed8b783cd097a252d6b85513f874c317219adb4f784fa56b2b8c
  Stored in directory: /root/.cache

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [ ]:
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from bertopic import BERTopic

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 1.83 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-10-03 08:29:24--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::3403:4be7, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bb

### 1. Improve

In [ ]:
import pandas as pd
topic_and_summary = pd.DataFrame(columns=['Topic', 'Summary', 'Topic_Text'])
topic_and_summary.loc[0] = ['a', 'b', 'c']
topic_and_summary

,Topic,Summary,Topic_Text
0,a,b,c


In [ ]:
topic_model = BERTopic(language="korean", calculate_probabilities=True, verbose=True, nr_topics =20)

In [ ]:
topics_im, probs_im = topic_model.fit_transform(improve_word)

Batches:   0%|          | 0/109 [00:00<?, ?it/s]

2022-10-03 08:43:54,482 - BERTopic - Transformed documents to Embeddings
2022-10-03 08:44:14,799 - BERTopic - Reduced dimensionality
2022-10-03 08:44:15,629 - BERTopic - Clustered reduced embeddings
2022-10-03 08:44:16,346 - BERTopic - Reduced number of topics from 66 to 21


In [ ]:
topic_info_im = topic_model.get_topic_info()
topic_info_im

,Topic,Count,Name
0,-1,1782,-1_결제_취소_유료_명상
1,0,274,0_무료_체험_결제_기간
2,1,189,1_설치_삭제_사용_유료
3,2,158,2_가입___
4,3,136,3_환불_결제_요청_신청
5,4,127,4_자동_결제_취소_최악
6,5,96,5_안드로이드_핸드폰_갤럭시_휴대폰
7,6,70,6_음악_배경음악_소리_유튜브
8,7,69,7_내야_내지_돈돈_비용
9,8,61,8_회원_탈퇴_가입_요청


In [ ]:
topic_model.generate_topic_labels()

['-1_결제_유료_소리',
 '0_무료_체험_줄알',
 '1_멋대로_보지_다시',
 '2_안드로이드_핸드폰_데이터',
 '3_프로도_찾기_제목',
 '4_설치_청구_삭제',
 '5_회원_탈퇴_가입',
 '6_탈퇴_처리_결제',
 '7_구독_계속_정지',
 '8_결제_자동_취소',
 '9_한국어_명상_나래',
 '10_매일_시간_기록',
 '11_삭제_도록_결제',
 '12_녹음_소리_오디오',
 '13_계정_삭제_구독',
 '14_음악_소리_수면',
 '15_정기_결제_중간',
 '16_환불_이메일_처리',
 '17_계정_탈퇴_삭제신청',
 '18_명상_마음_개인',
 '19_그래프_시간_사용']

In [ ]:
topic_model.visualize_barchart(top_n_topics=20)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [ ]:
improve_all = pd.DataFrame({'Topic':topics_im, 'Review_word': improve })
improve_all

,Topic,Review_word
10,-1,"이용권,얼마,소진,확인,요금,제,월간,플랜,주간,플랜,해주시"
21,-1,"제,카,이브,페이지,안보,오류,라면,수정"
27,0,"무료,이용,권총"
28,1,"시간,이지나,더,보고"
33,1,"타이머,정지"
...,...,...
7312,-1,"거원,안드로이드,버전,감지,마이크,가요"
7313,1,"업뎃후,실행,요"
7319,-1,"패치,이후,앱,실행"
7325,-1,"알람,진동,알람,설정,무조건,소리,알람,처럼,룸메,귀마개,저,아침,룸메,방해"


In [ ]:
improve_all.to_csv("improve_all.csv")

In [ ]:
improve_all = pd.read_csv("/content/drive/MyDrive/DSL/improve_all.csv")

In [ ]:
topic_doc = [] #전체 document와 토픽명이 들어있는 리스트
for i in range(len(topic_info_im['Topic'])):
  topic_doc.append(improve_all[improve_all.Topic== i])
  globals()[f'topic_{i}'] = topic_doc[i] #topic_0, topic_1 ~ 변수명 format 사용

topic_index = [] #topic 의 index가 담긴 list
for i in range(len(topic_info_im['Topic'])):
  topic_index.append(improve_all[improve_all.Topic== i].index)
  globals()[f'topic_{i}_index'] = topic_index[i]

for i in range(len(topic_info_im['Topic'])):
  globals()[f'topic_{i}_fullreview_im'] = reviews[topic_index[i]]
#topic_{i}_fullreview_im :improve 각 토픽별 전체 문장

In [ ]:
topic_0_fullreview_im

27          근데 무료 이용권 총 3회 있었는데 2회 뭐 때문인지 그냥 날 아가 있더라 구 용
106      마음이 편해 지내요 그리고 대부분 무료로 들을 수 있게 해 주셨으면 좋겠네요 이 ...
107      어른만 쓴다는 보장 규칙이 없으니 전체는 아니 여도 거의 다 무료 가입 안하고 들...
351                          유료 컨텐츠를 조금 무료로 조금만 바꿀 순 없을까요
545           유료에서 몇 가지를 무료로 만들던지 무료로 늘리던지 해 주시면 좋을 것 같아요
                              ...                        
5774                                   좀더 무료 화가 되었으면 좋겠어요
5850                                    그냥 공짜로 다해 주시면 좋은데
5873                       무료로 바탕 화면 나 가도 소리가 재생 되게 해 주세요
6223                                  무료 버전이 많아 졌음 좋켔 네 여
6714            무료 컨텐츠와 유료 컨텐츠의 경계가 애매한 데 공지를 더 명확히 해 주세요
Name: Review, Length: 122, dtype: object

####요약

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:
topic0_sen = '' 
for line in topic_0_fullreview_im:
  topic0_sen += line
  topic0_sen += '. '

In [ ]:
test = ' 개인적인 문 제일 수도 있지만 미라클 나 잇을 사용 할 때마다 휴대폰 진동에 머리가 아파서 잠을 못 자겠어요 계속 나오는 주파수 소리는 앱을 지웠다 까니까 사라졌네요.  이용권 얼마인가 보려고 한 개 소 진해서 확인 해 봤습니다 차라리 요 금제를 월간 플랜이나 주간 플랜으로 해 주시면 좋겠습니다.  또 한 테스트도 한번으로 하기에는 부족하다고.  월별 결제가 있었으면 좋겠어요.  계정을 탈퇴 하려고 했는데 된 건지 안된 건지 모르겠네요.  평소에 잘 썼는데 어제 갑자기 앱이 계속 중단 돼서 다시 켜고 또 중단 되고 구매한 부스터 몇 개를 날렸네요.  타이머 일 시 정지 할 수 있었으면 좋겠어요ㅠ 4시간 반보다 시간 늘려 줬으면 좋겠어요.  결제 오류 계속 떠서 못 쓰고 있어요.  사실 드라마틱하게 죽인다 쩐다 하는 반응은 아니구요.  결제 하려고 하면 계속 거래가 거부 되었다고 빨리 해결 해 주세요 떠요.  결제를 하려고 하는데 자꾸 거래가 거부 되었다고.'

In [ ]:
len(test)

486

In [ ]:
topic0_sen = "근데 무료 이용권 총 3회 있었는데 2회 뭐 때문인지 그냥 날 아가 있더라 구 용.마음이 편해 지내요 그리고 대부분 무료로 들을 수 있게 해 주셨으면 좋겠네요 이 점 고 처 주세요.  어른만 쓴다는 보장 규칙이 없으니 전체는 아니 여도 거의 다 무료 가입 안하고 들을 수 있게 해 줬으면 좋겠네요.  유료 컨텐츠를 조금 무료로 조금만 바꿀 순 없을까요.  유료에서 몇 가지를 무료로 만들던지 무료로 늘리던지 해 주시면 좋을 것 같아요.  ㅜ 다른 거 무료 말로 이 것만 무료로 다시 바꿔 주면 너무 좋겠어요.   근데 무료 이용권 총 3회 있었는데 2회 뭐 때문인지 그냥 날 아가 있더라 구 용.  마음이 편해 지내요 그리고 대부분 무료로 들을 수 있게 해 주셨으면 좋겠네요 이 점 고 처 주세요.  어른만 쓴다는 보장 규칙이 없으니 전체는 아니 여도 거의 다 무료 가입 안하고 들을 수 있게 해 줬으면 좋겠네요.  유료 컨텐츠를 조금 무료로 조금만 바꿀 순 없을까요.  유료에서 몇 가지를 무료로 만들던지 무료로 늘리던지 해 주시면 좋을 것 같아요.  ㅜ 다른 거 무료 말로 이 것만 무료로 다시 바꿔 주면 너무 좋겠어요.  실제로 마음이 편해 지는 느낌이에요 무료로 바뀐다면 모든 사람이 좋아할 거 같네요.  이어 플 설치하고 무료 체험 할 때 신중히 하세요 ㅡ 무료 체험 끝나면 바로 일 년 결 재가 되어서 울며 겨자 먹기로 일 년 사용 하게 되었어요.  무료 체험 뜨길래 뒤로 가기 했는데 자동 가입이 되네요.  무료 체험 후 오늘 결제 됐길래 앱스토어에서 정기 구독 취소 했는데 환불 처리 해 주세요 어디에다 요청 해야 하는지 어플에도 안 나와 있네요.  계정 탈퇴 앱에서 불가능 메일 로만 가능 처리 되었는지 확인 불가능 무료 체험을 하면 7일 후 이 자동 결제 되니 주의 하세요.  아니 무료 기능이 있어서 체험을 해보고 하는 것도 아니고 일단 결제 해 라면 띠 바 하.  오늘 무료 체험 하기 눌렀는데 무료 체험 끝나면 5만 9 천원 자동 결제 되는 거네요.  무료 체험 하기를 누르지 않았지만 자동 결제 될까 봐 글 남깁니다 never 탈퇴 원합니다.  저는 무료 체험을 하고 싶었는데 그 다음 날 새벽에 결제가 되었더라고요.  무료 체험 필요없으니 당장 탈퇴 시켜 주세요 그리고 자동 결제 안 되게 바랍니다.  무료 체험만 해보려고 깔아 봤어요 구독 취소 부탁드려요.  아이가 무료 체험 후 취소를 안해 자동적으로 결제가 되었습니다.  1 주일 무료 체험 후 결제 된다고 해서 뒤로 가기 눌렀는데 그냥 다음 화면으로 넘어가고 탈퇴 할 방법도 없네요.  무료 체험 이후 별도의 알림도 없이 오만원이 쑥 빠져나갔네요.  무료 체험 후 앱 사용 안 했는데 자동 결제 되었습니다.  그런 데 ㅠㅜ 일주일 무료 체험 후 59,000원 연간으로 안내 받았었는데 무료 체험 후 결제 취소 시켰다가 다시 연간 구독 하려고 보니 85,000원 연간으로 안 내 되네요.  아님 무료 체험 후 바로 이어서 결제를 안 하면 원래 저 가격인 거예요.  무료 체험만 해보려고 깔아 봤어요 아직 기간 남았는데 구독 취소가 잘 안 보입니다 구독 취소 부탁드려요.  사기 사이트 무료 체험 판 구경만 했는데 1년 자동 결제 되었습니다.  무료인 줄 알고 가입 했는데 며칠 후 유료로 자동 전환 되는 거 였나요 처리 부탁드려요.  이 거 무료 체험 되 서 7일 뒤에 결제 되는 거 아니겠죠 결제 안되게 해 주시고 계정 탈퇴 하게 해 주세요.  뭐야 무료로 쓰고 바로 자동 결제 되는 거지 같은 시스템이 어딨음 기분 나쁘니 계정 탈퇴 해 줘요.  무료 체험하고 자동 결재 취소도 없고 시기 어 플 계정 삭제 해 주세요.  무료 사용 기간 중 구독 취소 하려 니 정기 구독 중인 게 없다며 취소가 안되네요.  사용한 적도 없고 무료 체험 신청도 안 했으니 계정 삭제 해 주세요.  무료 체험인 줄 알고 신청 했어요 유료 전환 취소 부탁드립니다.  무료 체험 한번 해 봣 다가 구독 해제 할 려햇 는 데 ㅜ 결제가 되 버려 네 요 구독 취도하면 다시 환불 들어오는 거 맞죠.  무료로 운영 되는 중 알았는데 7일 무료 이후에는 1년 59,000 원씩 자동으로 결제 된다고 하는데 특별한 개인 동의 절차 없이 결제가 되는 것 같아 불쾌하여 취소 하러 고 합니다.  해서 가입 했는데 가입 누르니 무료 7일 후 바로 연간 이용 료가 자동 결제 됩니다. "

In [ ]:
len(topic0_sen)

2144

In [ ]:
topic0_sen[:1000]

'근데 무료 이용권 총 3회 있었는데 2회 뭐 때문인지 그냥 날 아가 있더라 구 용.마음이 편해 지내요 그리고 대부분 무료로 들을 수 있게 해 주셨으면 좋겠네요 이 점 고 처 주세요.  어른만 쓴다는 보장 규칙이 없으니 전체는 아니 여도 거의 다 무료 가입 안하고 들을 수 있게 해 줬으면 좋겠네요.  유료 컨텐츠를 조금 무료로 조금만 바꿀 순 없을까요.  유료에서 몇 가지를 무료로 만들던지 무료로 늘리던지 해 주시면 좋을 것 같아요.  ㅜ 다른 거 무료 말로 이 것만 무료로 다시 바꿔 주면 너무 좋겠어요.   근데 무료 이용권 총 3회 있었는데 2회 뭐 때문인지 그냥 날 아가 있더라 구 용.  마음이 편해 지내요 그리고 대부분 무료로 들을 수 있게 해 주셨으면 좋겠네요 이 점 고 처 주세요.  어른만 쓴다는 보장 규칙이 없으니 전체는 아니 여도 거의 다 무료 가입 안하고 들을 수 있게 해 줬으면 좋겠네요.  유료 컨텐츠를 조금 무료로 조금만 바꿀 순 없을까요.  유료에서 몇 가지를 무료로 만들던지 무료로 늘리던지 해 주시면 좋을 것 같아요.  ㅜ 다른 거 무료 말로 이 것만 무료로 다시 바꿔 주면 너무 좋겠어요.  실제로 마음이 편해 지는 느낌이에요 무료로 바뀐다면 모든 사람이 좋아할 거 같네요.  이어 플 설치하고 무료 체험 할 때 신중히 하세요 ㅡ 무료 체험 끝나면 바로 일 년 결 재가 되어서 울며 겨자 먹기로 일 년 사용 하게 되었어요.  무료 체험 뜨길래 뒤로 가기 했는데 자동 가입이 되네요.  무료 체험 후 오늘 결제 됐길래 앱스토어에서 정기 구독 취소 했는데 환불 처리 해 주세요 어디에다 요청 해야 하는지 어플에도 안 나와 있네요.  계정 탈퇴 앱에서 불가능 메일 로만 가능 처리 되었는지 확인 불가능 무료 체험을 하면 7일 후 이 자동 결제 되니 주의 하세요.  아니 무료 기능이 있어서 체험을 해보고 하는 것도 아니고 일단 결제 해 라면 띠 바 하.  오늘 무료 체험 하기 눌렀는데 무료 체험 끝나면 5만 9 천원 자동 결제 되는 거네요.  무료 체험

In [ ]:
raw_input_ids = tokenizer.encode(topic0_sen)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
input_ids = torch.tensor([input_ids])
summary_text_ids = model.generate(
    input_ids=input_ids,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    length_penalty=1.0, # 길이에 대한 penalty값. 1보다 작은 경우 더 짧은 문장을 생성하도록 유도하며, 1보다 클 경우 길이가 더 긴 문장을 유도
    max_length=128,     # 요약문의 최대 길이 설정
    min_length=16,      # 요약문의 최소 길이 설정
    num_beams=4 ,       # 문장 생성시 다음 단어를 탐색하는 영역의 개수 
)
topic15_summ = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

IndexError: ignored

In [ ]:
tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

NameError: ignored

### 3. Good

In [ ]:
topics, probs = topic_model.fit_transform(good_word)

Batches:   0%|          | 0/102 [00:00<?, ?it/s]

2022-10-02 05:19:32,718 - BERTopic - Transformed documents to Embeddings
2022-10-02 05:19:59,142 - BERTopic - Reduced dimensionality
2022-10-02 05:20:00,090 - BERTopic - Clustered reduced embeddings
2022-10-02 05:20:00,942 - BERTopic - Reduced number of topics from 56 to 21


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name
0,-1,1776,-1_소리_마음_명상_집중
1,0,303,0_광고_최고_정말_사용
2,1,130,1_무료_결제_유료_버전
3,2,100,2_소리_녹음_여러가지_파도
4,3,99,3_잠도_수면_꿀잠_평소
5,4,73,4____
6,5,72,5_도움_마음_안정_프로
7,6,67,6_음악_자연_기분_기능
8,7,65,7_아침_시작_하루_알람
9,8,63,8_아기_아들_평소_금방


In [ ]:
good_all = pd.DataFrame({'Topic':topics, 'Review_word':good })
good_all

,Topic,Review_word
0,4,
2,7,"가끔,아침,머리,몽롱,데땐,아침밥,커피한잔,하루,시작"
16,-1,"항상,가중,매일,꿈,푹,느낌,잘못,앱,사용,후,꿈,번안,푹"
17,-1,"새벽,한번,소리,집중,느낌"
18,-1,"분간,명상,후,모노클,비트,항목,시간,이오,사람,명상,몸,긴장,보"
...,...,...
7330,-1,"기능,도지,조율,유료,어플,값"
7331,-1,"생각,극적,효과,그냥,정시,알람,느낌"
7332,-1,"침대,폰,머리,에두,제일,측정,발치,도정,측정"
7333,4,


In [ ]:
topic_model.visualize_barchart(top_n_topics=20)

In [ ]:
good_all.to_csv("good_all.csv")

In [ ]:
good_doc = [] #전체 document와 토픽명이 들어있는 리스트
for i in range(len(topic_info['Topic'])):
  good_doc.append(good_all[good_all.Topic== i])
  globals()[f'topic_{i}'] = topic_doc[i] #topic_0, topic_1 ~ 변수명 format 사용

topic_index = [] #topic 의 index가 담긴 list
for i in range(len(topic_info['Topic'])):
  topic_index.append(good_all[good_all.Topic== i].index)
  globals()[f'topic_{i}_index'] = topic_index[i]

for i in range(len(topic_info['Topic'])):
  globals()[f'topic_{i}_fullreview'] = reviews[topic_index[i]]
#topic_{i}_fullreview : 각 토픽별 전체 문장

In [ ]:
topic_0_fullreview

22         원래 새벽에 5-6 번 정도 깨는데 이 앱을 사용하고는 한 번도 안 깨고 푹 잤어요
58                    앱을 사용한 첫 날 그렇게 잠을 못 자 던 제가 꿀 잠을 잤어요
60       유 튜브에 명상 음악도 많고 좋은 강연도 많지만 코끼리 앱은 특별한 분위기와 편안...
78                            불안한 저의 마음을 안정 시키는 감사한 앱 입니다
87       글을 쓰거나 책 볼 때 들으니 집중이 잘 되 요 이렇게 종은 앱을 소개 해 주셔서...
                              ...                        
6998                                앱 자체는 나쁘지 않다는 생각이 듭니다
7093         알람 소리가 좋고 이 앱을 사용 하고부터 기상 시간이 조금 개운 해진 것 같아요
7202     좋은 앱이고 자동 결제 되는 거는 본인이 이 앱 깔면서 잘 읽어 봣으면 다 알 수...
7243                  그간의 알람 앱들과 달리 기분 나쁘지 않게 기상 할 수 있었네요
7308                                 지각을 면 하기 위해 이 앱을 씁니다
Name: Review, Length: 303, dtype: object

### 4. Personal

In [ ]:
topics, probs = topic_model.fit_transform(personal_word)

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

2022-10-02 06:35:16,594 - BERTopic - Transformed documents to Embeddings
2022-10-02 06:35:21,408 - BERTopic - Reduced dimensionality
2022-10-02 06:35:21,448 - BERTopic - Clustered reduced embeddings
2022-10-02 06:35:21,478 - BERTopic - Reduced number of topics from 3 to 3


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name
0,0,18,0_질문_목록__
1,1,15,1_피키_캐스트_살림_만남
2,2,582,2_결제_명상_사용_마음


In [ ]:
personal_all = pd.DataFrame({'Topic':topics, 'Review_word':personal })
personal_all

,Topic,Review_word
1,2,"처음,휴대폰,소리,적응,머리,이틀,적응,아집"
4,2,"앱,표시,광고,대한,평가,란,신경안,때,만사,앱,광고,평가,란,보고,처음"
5,2,"워치,번,점수,기록"
6,2,"꿈결,집,진동,알람,땜,앱,소리,뭐,묘아,직,베타,단계"
11,2,"오류,버그,중지,이용권,다시,드릴,필요,이용자,입장,버그,문제,이용권,문제,월간,플..."
...,...,...
7203,2,"기와,달,무료,체험,후연"
7206,2,"거미,리,결제,체험,시작,결제,임"
7229,2,"전자파,몸,머리,직빵,생각"
7255,2,"무료,체험,안,건너,기하,알람,기능,사용"


In [ ]:
personal_all.to_csv("personal_all.csv")

# Reference

- https://maartengr.github.io/BERTopic/index.html
- https://arxiv.org/abs/2203.05794
- https://github.com/SKTBrain/KoBERT
- https://huggingface.co/gogamza/kobart-summarization